In [36]:
from pyspark.sql.types import StructField, StructType, IntegerType, StringType
from pyspark.sql import functions as f

StatementMeta(tcccibele, 4, 36, Finished, Available)

In [37]:
# Definir schema do arquivo
schema = StructType([ 
    StructField("None", StringType(), True), 
    StructField("FaixaEtariaSuspeito", StringType(), True), 
    StructField("Janeiro", StringType(), True), 
    StructField("Fevereiro", StringType(), True), 
    StructField("Março", StringType(), True), 
    StructField("Abril", StringType(), True), 
    StructField("Maio", StringType(), True), 
    StructField("Junho", StringType(), True), 
    StructField("Julho", StringType(), True), 
    StructField("Agosto", StringType(), True), 
    StructField("Setembro", StringType(), True), 
    StructField("Outubro", StringType(), True), 
    StructField("Novembro", StringType(), True), 
    StructField("Dezembro", StringType(), True), 
    StructField("Total", StringType(), True)
])

StatementMeta(tcccibele, 4, 37, Finished, Available)

In [38]:
# Ler arquivo de origem da landing zone
df = spark.read.format("csv") \
            .schema(schema) \
            .options(quote='"') \
            .load('/landing_zone/2019/Balanco-2019-Ligue180-Idade Suspeito.csv')

StatementMeta(tcccibele, 4, 38, Finished, Available)

In [39]:
# Remover colunas e linhas sem valor para o data lake
df = df.where((df[1].isNotNull()) \
                & (df[1] != 'Total') \
                & (df[-1] != 'Total 2019') \
                & (df[-1].isNotNull()))
df = df.drop('None')
df = df.drop('Total')

StatementMeta(tcccibele, 4, 39, Finished, Available)

In [40]:
# Remover "," de todos os campos e substitui null por 0
names = df.schema.names
for name in names:
    if name == 'FaixaEtariaSuspeito':
        continue
    df = df.withColumn(name,f.regexp_replace(name,',',''))
    df = df.withColumn(name, df[name].cast(IntegerType()))

df = df.fillna(0)

StatementMeta(tcccibele, 4, 40, Finished, Available)

In [41]:
# Escrever dados limpos em formato parque no azure storage
df.write.format('parquet').mode('overwrite').save('/history_zone/2019/IdadeSuspeito/')

StatementMeta(tcccibele, 4, 41, Finished, Available)

In [42]:
df = spark.read.parquet('/history_zone/2019/IdadeSuspeito/')
df.toPandas()

StatementMeta(tcccibele, 4, 42, Finished, Available)

,FaixaEtariaSuspeito,Janeiro,Fevereiro,Março,Abril,Maio,Junho,Julho,Agosto,Setembro,Outubro,Novembro,Dezembro
0,0 a 3 anos,0,0,0,0,0,0,0,0,0,0,0,0
1,4 a 7 anos,0,1,0,1,0,0,0,0,0,1,0,0
2,8 a 11 anos,1,4,2,1,0,0,1,1,2,2,1,0
3,12 a 14 anos,21,18,24,15,14,16,12,16,16,11,12,4
4,15 a 17 anos,82,70,59,70,55,67,63,46,50,43,45,21
5,18 a 24 anos,923,674,751,703,768,564,647,715,633,704,654,349
6,25 a 30 anos,1415,1164,1286,1107,1144,994,996,1172,1127,1163,1079,540
7,31 a 35 anos,1177,946,1055,959,888,825,831,897,962,907,858,464
8,36 a 40 anos,1378,1047,1264,1062,1059,964,962,1125,1106,1111,1035,532
9,41 a 45 anos,901,770,870,724,747,640,647,763,724,687,698,335
